Install all necessary packages.

In [ ]:
from PIL import Image as im
import tensorflow as tf
from tensorflow.keras import datasets, layers, Sequential, models
import numpy as np

Set all initial values.

In [ ]:
train_data_points = 1600
test_data_points = 300
total_data_points = train_data_points + test_data_points
pixels = 50
bins = 62
mid_layer = 750
training_iters = 10
learning_rate = 2e-3
batch_size = 512
num_epochs = 1
lr_reduction_factor = 0.5

Function that reduces learning rate for every epoch.

In [ ]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * lr_reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

Converts machine output to human-readable output.

In [ ]:
def convert_to_char(value):
    if -1 < value < 10:
        return chr(int(value))
    elif 9 < value < 36:
        return chr(int(value + 55))
    elif 35 < value < 62:
        return chr(int(value + 61))
    else:
        return 'Invalid input for convert_to_char()'

Accept images as input and create an array of images

In [ ]:
images = []
folder_path = "C:/NN/NIST SD 19/DataSet/"
for i in range(48, 123):
    if 47 < i < 58 or 64 < i < 91 or 96 < i < 123:
        for j in range(total_data_points):
            total_iterations = bins * total_data_points
            p = -1
            if 47 < i < 58:
                p = np.round((((i - 48) * total_data_points + j) / total_iterations) * 100, 2)
            elif 64 < i < 91:
                p = np.round((((i - 55) * total_data_points + j) / total_iterations) * 100, 2)
            elif 96 < i < 123:
                p = np.round((((i - 61) * total_data_points + j) / total_iterations) * 100, 2)
            print('\r''Data loading:', p, '%', end='')

            filename = folder_path + str(i) + "-" + str(j) + ".png"
            img = im.open(filename)
            img = img.resize((pixels, pixels))
            img = img.convert('L')
            img_array = np.asarray(img)
            images.append(img_array)
            img.close()
            img.close()

print('\n')

Reshapes the images array into a numpy array and creates specific training labels for each image.

In [ ]:
images_array = np.reshape(images, (bins, total_data_points, pixels, pixels))

training_data = images_array[:, 0:train_data_points, :, :]
training_data = np.reshape(training_data, (train_data_points * bins, pixels, pixels))

test_data = images_array[:, train_data_points:total_data_points, :, :]
test_data = np.reshape(test_data, (test_data_points * bins, pixels, pixels))

training_labels = np.empty(0)
test_labels = np.empty(0)
for i in range(bins):
    to_add = i * np.ones(train_data_points)
    training_labels = np.append(training_labels, to_add)
    to_add = i * np.ones(test_data_points)
    test_labels = np.append(test_labels, to_add)

training_data = training_data / 255.0
test_data = test_data / 255.0

Creates the CNN model.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(pixels, (3, 3), activation='relu', input_shape=(pixels, pixels, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(2 * pixels, activation='relu'))
model.add(layers.Dense(62))

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

Compiles the CNN and gives accuracy as output.

In [ ]:
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(training_data, training_labels, callbacks=[LearningRateReducerCb()], epochs=num_epochs,
                    validation_data=(test_data, test_labels))

test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print("Accuracy = " + str(np.round(test_acc * 100, 3)) + " %")